# Un mapa interactivo de casos de COVID-19 

David Sánchez

<p style='text-align: justify; font-size:12px;'>
    Tiempo después del inicio del estado de emergencia en Perú, la plataforma de <a href="https://www.datosabiertos.gob.pe/">datos abiertos</a> del país empezó a hacer pública la información asociada a los casos positivos de COVID-19, así como de las muertes atribuibles a la enfermedad. Aunque esta no recoge muchas características sobre las personas contagiadas o fallecidas, igualmente es posible encontrarle alguna aplicación.
<br>
<br>
    Recientemente he visto un ejercicio en donde se elabora un mapa sobre la ejecución del presupuesto público a nivel distrital con <a href="https://leafletjs.com/">Leaflet</a>, una librería open-source basada en JavaScript. Sin embargo, en el ejercicio noté que los autores no logran redirigir la información directamente desde el proveedor hacia el software que utilizaron (R). Esto redunda en dos cosas: 1) que los resultados no puedan ser rápidamente replicables porque la información del proveedor se ha manejado a nivel local, y 2) que los resultados sean difíciles de actualizar porque una parte del proceso no ha sido "automatizada". Y estas son cosas que no queremos, la que queremos es correr un código y ya; y que si la información de origen cambia, nuestros resultados también lo hagan.
<br>
<br>
    Así, voy a realizar un ejercicio similar que supere esos incovenientes utilizando la información de la plataforma de datos abiertos, y generando un mapa de Leaflet en donde se muestren los contagiados de COVID-19 por distrito a través de Python. Asimismo, esta aplicación tendrá la característica de que podrá recuperar información directamente desde la plataforma de origen, por lo que en el proceso resuelve las cuestiones anteriores.
    </p>

### Paso 1. Selenium

<p style='text-align: justify; font-size: 12px'>La plataforma de datos abiertos brinda un API solo para algunas de las bases de datos. La información sobre los casos de COVID-19 no puede ser recuperada por esa vía; solo a través de un link que abre una ventana de diálogo para la descarga. Esta es una primera dificultad, pero podemos abordarla a través de Selenium.

1.1. Paquetes y paths
<p style='text-align: justify; font-size: 12px'>
    <code>path</code>, una carpeta donde guardar la data a nivel local.
    <br>
<code>link</code>, dirección donde se encuentra el link de descarga. 
    <br>
<code>xpath</code>, XPATH del link de descarga en <code>link</code>.
    </p>

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import time

path = r'C:\Users\RODRIGO\Desktop\MinsaData'
link = 'https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa'
xpath = '//*[@id="data-and-resources"]/div/div/ul/li/div/span/a'

2.2. Configuración y descarga

<p style='text-align: justify; font-size:12px;'>
    Selenium permitirá simular un navegador; en consecuencia, podemos superar la ventana de diálogo y recuperar la información directamente del proveedor para almacenarla a nivel local. Esta parte es la que hace posible automatizar el proceso aunque la plataforma de datos abiertos no tenga API. Aunque la información cambie, siempre podremos acceder para descargar la información más reciente y guardarla a nivel local.
<br>
<br>
    Como simularemos un navegador (Chrome), existirá un tiempo de descarga. Es decir, el código en Python seguirá corriendo, mientras que sucede la descarga. Para solucionar esto, agrego un tiempo de 2 minutos, tiempo prudente para que nuestro navegador termine de descargar los archivos desde la <a href="https://cloud.minsa.gob.pe/">nube del MINSA</a> y que esta se almacene en local.
    </p>

In [2]:
options = webdriver.ChromeOptions()
preferences = {'download.default_directory': path, 'safebrowsing.enabled': 'false'}

options.add_experimental_option ('prefs', preferences)

driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

driver.get(link)
driver.find_element(By.XPATH, xpath).click()

x = int()
while x < 120: 
    time.sleep(30)
    x += 30
    print('Tiempo restante:', 120-x, 'segundos')

driver.close()
print('Descarga completa.')

Tiempo restante: 90 segundos
Tiempo restante: 60 segundos
Tiempo restante: 30 segundos
Tiempo restante: 0 segundos
Descarga completa.


### Paso 2. Pandas

**2.1. CSV: Contagiados y problemas**

<p style='text-align: justify; font-size:12px;'>
    Una vez almacenada la información localmente, vemos que este archivo es un CSV. Este archivo tiene dos detalles importantes que se convierten en problemas; los abordaremos separadamente.
    </p>

In [3]:
import pandas as pd
import os

path = r'C:\Users\RODRIGO\Desktop\MinsaData'
os.chdir(path)

dataFile = pd.read_csv('positivos_covid.csv', sep=',', encoding='ISO-8859-1')
dataFile.head(3)

,FECHA_CORTE,UUID,DEPARTAMENTO,PROVINCIA,DISTRITO,METODODX,EDAD,SEXO,FECHA_RESULTADO
0,20200915,7320cabdc1aaca6c59014cae76a134e6,LIMA,LIMA,SANTA ANITA,PCR,49,MASCULINO,20200701
1,20200915,e81602051997ace8340bb8c18fe24c65,LIMA,EN INVESTIGACIÓN,EN INVESTIGACIÓN,PCR,49,MASCULINO,20200701
2,20200915,71ecb6bccb248b0bb2ac72ed51b5e979,LIMA,LIMA,PUEBLO LIBRE,PCR,47,MASCULINO,20200706


*2.1.1. Problema 1: Lima y Lima-Región*

<p style='text-align: justify; font-size:12px;'>
    Separaron el departamento de Lima en dos: Lima y Lima-Región. Para efectos de encontrar información georreferenciada, eso no es bueno. Agruparemos ambos.
    </p>

In [4]:
dataFile.loc[dataFile['DEPARTAMENTO'] == 'LIMA REGION', 'DEPARTAMENTO'] = 'LIMA'
print('Corregido.')

Corregido.


*2.1.2. Problema 2: Casos "EN INVESTIGACIÓN"*

<p style='text-align: justify; font-size:12px;'>
    En algunos departamentos al parecer no se sabe exactamente a qué provincia o distrito pertenece la persona contagiada. Estas observaciones no se contarán porque no podemos identificarlas a nivel distrital.
    </p>

In [5]:
for x in ['PROVINCIA', 'DISTRITO']:
    condition = dataFile[x] == 'EN INVESTIGACIÓN'
    dataFile = dataFile[~condition]
print('Corregido.')

Corregido.


**2.1. Ubicación geográfica: Distritos y coordenadas**

<p style='text-align: justify; font-size:12px;'>
    La información sobre coordenadas geográficas de los distritos la recupero a través de un shapefile de <a href="https://www.geogpsperu.com/2014/03/base-de-datos-peru-shapefile-shp-minam.html">geogpsperu.com</a>. Esta información la almaceno en un archivo CSV y se encuentra en el repositorio.

In [6]:
coor_link = 'https://raw.githubusercontent.com/DRodrigo96/SomeProjects/master/Contagio%20COVID-19/Coordenadas/COORDENADAS%20DISTRITAL.csv'
shapef = pd.read_csv(coor_link, sep=';', encoding='utf-8-sig')
print('Coordenadas cargadas.')

Coordenadas cargadas.


### Paso 3: Fuzzy Wuzzy

<p style='text-align: justify; font-size:12px;'>
    Si bien en la información sobre contagios que brinda datosabiertos.gob.pe se encuentran observaciones a nivel de distrito, no existe un identificador único de este (ubigeo). La única forma de realizar un matching entre la información de contagios y las coordenadas distritales es a través del nombre de los departamentos, provincias y distritos.
    </p>

In [7]:
dataFile.sort_values(by=['DEPARTAMENTO','PROVINCIA','DISTRITO'], inplace=True)
dataFile['INDEX'] = list(zip(dataFile['DEPARTAMENTO'], dataFile['PROVINCIA'], dataFile['DISTRITO']))

shapef.sort_values(by=['DEPARTAMENTO','PROVINCIA','DISTRITO'], inplace=True)
shapef['INDEX'] = list(zip(shapef['DEPARTAMENTO'], shapef['PROVINCIA'], shapef['DISTRITO']))

dataFile_index = list(dataFile['INDEX'].unique())
shapef_index = list(shapef['INDEX'].unique())

not_in_shp = list()
for x in shapef_index:
    if x not in dataFile_index:
        not_in_shp.append(x)

print('Número de distritos con nombre diferente:', len(not_in_shp) - (len(shapef_index) - len(dataFile_index)))

Número de distritos con nombre diferente: 19


<p style='text-align: justify; font-size:12px;'>
    Entonces, hay 19 distritos que tienen el nombre escrito de forma diferente en alguna de las bases; pero en realidad sí se refieren al mismo. Esto significa que no existiría un matching perfecto si no hiciéramos ajustes en los nombres. Para superar esto, usaremos una combinación entre Fuzzy Wuzzy y la correción manual donde sea necesario.

**3.1. String matching**

<p style='text-align: justify; font-size:12px;'>
    En términos sencillos, Fuzzy Wuzzy permite obtener un score que indica qué tan similares son dos cadenas de caracteres, por ejemplo:

In [9]:
from fuzzywuzzy import fuzz

x = fuzz.ratio("LIMA PORTILLO MANANTAY", 'LIMA PORTILLO MANANTAI')
if x > 80:
    print("Score: {}. It's a match!".format(x))

Score: 95. It's a match!


<p style='text-align: justify; font-size:12px;'>
    Con esta herramienta podemos hacer nombres homogéneos entre las bases para luego hacer un matching adecuado.

*3.1.1. Corrección con threshold de score 95*

<p style='text-align: justify; font-size:12px;'>
    Inicialmente usaremos Fuzzy Wuzzy para automatizar la correción de nombres.

In [10]:
for x, y, z in not_in_shp:
    for a, b, c in dataFile_index:
        ratio = fuzz.ratio(str(y + ' ' + z), str(b + ' ' + c))
        if ratio >= 95:
            dataFile.loc[dataFile['PROVINCIA'] == b, 'PROVINCIA'] = y
            dataFile.loc[dataFile['DISTRITO'] == c, 'DISTRITO'] = z
        else:
            pass

*3.1.2. Correción manual de nombres*

<p style='text-align: justify; font-size:12px;'>
    Algunos de los nombres no superan el threshold de 95 puntos, pero igualmente se refieren al mismo distrito. Estos necesariamente requieren ser ajustados manualmente. Esta situación podría ser evitada si las instituciones públicas manejaran un nombre homogéneo entre las provincias y distritos. Corriendo el código anterior para un <code>ratio</code> mayor o igual a 80 se puede recuperar una lista de nombres, y entre ellos podemos identificar cuáles se deben ajustar.

In [11]:
dataFile.loc[dataFile['PROVINCIA'] == 'NAZCA', 'PROVINCIA'] = 'NASCA'
dataFile.loc[dataFile['DISTRITO'] == 'HUAY HUAY', 'DISTRITO'] = 'HUAY-HUAY'
dataFile.loc[dataFile['DISTRITO'] == 'SAN FCO DE ASIS DE YARUSYACAN', 'DISTRITO'] = 'SAN FRANCISCO DE ASIS DE YARUSYACAN'
dataFile.loc[dataFile['DISTRITO'] == 'SONDOR', 'DISTRITO'] = 'SONDORILLO'
dataFile.loc[dataFile['DISTRITO'] == 'CORONEL GREGORIO ALBARRACIN L.', 'DISTRITO'] = 'CORONEL GREGORIO ALBARRACIN LANCHIPA'
dataFile.loc[dataFile['DISTRITO'] == 'NAZCA', 'DISTRITO'] = 'NASCA'
dataFile.loc[dataFile['DISTRITO'] == 'ANDRES AVELINO CACERES D.', 'DISTRITO'] = 'ANDRES AVELINO CACERES DORREGARAY'

dataFile['INDEX'] = list(zip(dataFile['DEPARTAMENTO'], dataFile['PROVINCIA'], dataFile['DISTRITO']))

dataFile_index = list(dataFile['INDEX'].unique())
shapef_index = list(shapef['INDEX'].unique())

not_in_shp = list()
for x in shapef_index:
    if x not in dataFile_index:
        not_in_shp.append(x)

print('Número de distritos con nombre diferente:', len(not_in_shp) - (len(shapef_index) - len(dataFile_index)))

Número de distritos con nombre diferente: 0


<p style='text-align: justify; font-size:12px;'>
    Hemos arreglado este incoveniente. Nuevamente, las instituciones públicas deberían todas manejar no solo el identificador único de distrito (ubigeo); sino tambien un nombre uniforme entre sus bases de datos.

### Paso 4: Información Georreferenciada

<p style='text-align: justify; font-size:12px;'>
    La información de contagios se encuentra a nivel de observación, por lo que la agruparemos a nivel distrital; esto redunda en simplemente contar cuántas observaciones caen en un distrito. Adicionalmente, recuperaremos la edad promedio en cada uno de ellos. Luego haremos un join utilizando como index el nombre del departamento, provincia y distrito. Es por esto que homogeneizamos los nombres.

In [13]:
# Collapse información a nivel de distrito
collapseData = dataFile.groupby(['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO']).agg({'FECHA_CORTE': 'count', 'EDAD': 'mean'})
collapseData.reset_index(inplace=True)

# collapseData nuevo index
collapseData['INDEX'] = collapseData[['DEPARTAMENTO','PROVINCIA', 'DISTRITO']].agg(' '.join, axis=1) 
collapseData.set_index('INDEX', inplace=True)
collapseData.drop(['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO'], axis=1, inplace=True)

# shapef nuevo index
shapef['INDEX'] = shapef[['DEPARTAMENTO', 'PROVINCIA', 'DISTRITO']].agg(' '.join, axis=1)
shapef.set_index('INDEX', inplace=True)

# Join de dataframes
dfGeoref = shapef.join(collapseData, how='left')

# Drop distritos sin datos
dfGeoref.dropna(inplace=True)

### Paso 5: Folium: Python, JavaScript, CSS & HTML

<p style='text-align: justify; font-size:12px;'>
    Lo anterior será relativamente sencillo cuando nos adentremos en este paso. Aquí diseñaremos nuestro mapa interactivo. Aplicaremos el paquete Folium, algo de JavaScript, CSS y HTML para lograr un diseño interesante.

**5.1. Paquetes y variables**

In [14]:
import folium
from folium import Map, Marker
from folium import plugins
from folium.plugins import MarkerCluster
from PIL import ImageFont
import json
from jinja2 import Template

latitude = dfGeoref['LATITUD']
longitude = dfGeoref['LONGITUD']
distrito = dfGeoref['DISTRITO']
provincia = dfGeoref['PROVINCIA']
departamento = dfGeoref['DEPARTAMENTO']
numero = dfGeoref['FECHA_CORTE']
edades = dfGeoref['EDAD']

**5.2. JavaScript en acción**

<p style='text-align: justify; font-size:12px;'>
    En nuestro mapa queremos mostrar el número de contagios por área, por lo que para modificar ese aspecto de forma dinámica debemos utilizar un poco de JavaScript y Python en conjunto. Gracias a <a href="https://stackoverflow.com/users/1375553/vadim-gremyachev">Vadim Gremyachev</a>, de quien recuperé el código inicial de JavaScript y Python que luego fue adaptado.

In [15]:
class MarkerWithProps(Marker):
    _template = Template(u"""
        {% macro script(this, kwargs) %}
        var {{this.get_name()}} = L.marker(
            [{{this.location[0]}}, {{this.location[1]}}],
            {
                icon: new L.Icon.Default(),
                {%- if this.draggable %}
                draggable: true,
                autoPan: true,
                {%- endif %}
                {%- if this.props %}
                props : {{ this.props }} 
                {%- endif %}
                }
            )
            .addTo({{this._parent.get_name()}});
        {% endmacro %}
        """)
    def __init__(self, location, popup=None, tooltip=None, icon=None,
                 draggable=False, props = None ):
        super(MarkerWithProps, self).__init__(location=location,popup=popup,tooltip=tooltip,icon=icon,draggable=draggable)
        self.props = json.loads(json.dumps(props))    

        
icon_create_function = """
    function(cluster) {

    var c = ' marker-cluster-';

    var markers = cluster.getAllChildMarkers();
    var sum = 0;
    for (var i = 0; i < markers.length; i++) {
        sum += markers[i].options.props.population;
    }
    var sum_total = sum;

    if (sum_total < 1600) {
        c += 'small';
    } else if (sum_total < 6000) {
        c += 'medium';
    } else {
        c += 'large';
    }

    return new L.DivIcon({ html: '<div><span style="font-size: 7pt">' + sum_total + '</span></div>', className: 'marker-cluster' + c, iconSize: new L.Point(40, 40) });
    
    }
    """

**5.3. Mapa en Folium**

<p style='text-align: justify; font-size:12px;'>
    Aquí generaré un mapa a través de Folium, para el cual usaré el diseño "Matrix" de <a href="https://www.jawg.io/en/">JawgMaps</a>. Es necesario un token de a <a href="https://www.jawg.io/lab/">JawgLab</a> para usarlo; sin embargo dejaré un diseño alternativo en caso no se tenga un token para que el código continúe sin errores.

In [16]:
# Coordenadas de Perú
coor_PE = (-8.043040, -75.534517)

try:
    usedTile = 'https://tile.jawg.io/jawg-matrix/{z}/{x}/{y}{r}.png?access-token='+'{token}'.format(token=open("JAWGMAP API KEY.txt", 'r').read())
except:
    usedTile = 'https://tiles.stadiamaps.com/tiles/alidade_smooth_dark/{z}/{x}/{y}{r}.png'

# Mapa base
maPE = Map(
    coor_PE, 
    zoom_start = 5, 
    tiles=usedTile, 
    attr='''
    <a href="https://www.jawg.io/en/">JawgMaps</a>. 
    Fuente: <a href="https://www.datosabiertos.gob.pe/">datosabiertos.gob.pe</a>.
    Elaboración: <a href="https://www.linkedin.com/in/rodrigosanchezn/">David Sánchez</a>.
    '''
    )

# Marcadores en el mapa
contagio = MarkerCluster(icon_create_function=icon_create_function).add_to(maPE)
font = ImageFont.truetype('times.ttf', 12)

def colorcode(x):
    if x in range(0,1600):
        color = 'blue'
        icon = 'heartbeat'
    elif x in range(1601,6000):
        color = 'orange'
        icon = 'exclamation-circle'
    else:
        color = 'red'
        icon = 'fa-ambulance'
    return (color, icon)

for lat, lon, dep, pro, dis, num, edad in zip(latitude,longitude, departamento, provincia, distrito, numero, edades):
    contagio.add_child(MarkerWithProps(
        location=[lat,lon],
        icon=folium.Icon(color=colorcode(num)[0], icon=colorcode(num)[1], prefix='fa'),
        popup = folium.Popup(
            html='{}, {}, {}<br>Contagiados: {}<br>Edad promedio: {}'.format(dep,pro,dis,int(num),round(edad)),
            max_width=font.getsize(dep+pro+dis+' '*4)[0], min_width=font.getsize(dep+pro+dis+' '*4)[0],
            sticky=True),
        tooltip=folium.Tooltip('{}, {}, {}<br>Contagiados: {}<br>Edad promedio: {}'.format(dep,pro,dis,int(num),round(edad))),
        props = { 'population': num}
    ))

**5.4. CSS & HTML para leyendas**

<p style='text-align: justify; font-size:12px;'>
    Uso el siguiente código para añadir leyendas personalizadas en nuestro mapa. Gracias a <a href="https://stackoverflow.com/users/7084278/inlaw">InLaw</a>, de quien recuperé la idea sobre cómo añadir estos objetos al mapa.

In [17]:
item_txt = """<br> &nbsp; <i class="fa fa-map-marker fa-2x" style="color:{col}"></i> &nbsp; {item}"""
item_clu = """<br> &nbsp; <i class="fa fa-circle-o fa-lg" aria-hidden="true" style="color:{col}"></i> &nbsp; {item}"""
itms_1 = item_txt.format(item="Menos de 1600", col="#82CAFA")
itms_2 = item_txt.format(item="Entre 1600 y 6000", col="orange")
itms_3 = item_txt.format(item="Más de 6000", col="red")
itms_4 = item_clu.format(item="Contagios en el área", col="green")

legend_html = '''
    <div style="
    position: fixed; 
    top: 120px; left: 20px; width: 150px; height: 100px; margin:0 auto;
    border:2px solid grey; z-index:9999; 
    background-color:white;
    opacity: .55;
    font-size:10px;
    font-weight: bold;
    line-height: 12px
    "> 
    &nbsp; 
    {itm_txt_4}
    {itm_txt_1}
    {itm_txt_2}
    {itm_txt_3}
    </div> '''.format(itm_txt_1=itms_1, itm_txt_2=itms_2, itm_txt_3=itms_3, itm_txt_4=itms_4)

title_html = '''
    <div style="
    position: fixed; 
    top: 80px; left: 10px; width: 260px; 
    height: 35px; line-height: 35px; text-align: center;
    border:2px solid grey; z-index:9999; 
    border-radius: 25px;
    background-color:white;
    opacity: .55;
    font-size:12px;
    font-family: fantasy; 
    "> 
    {title}
    </div> '''.format(title="COVID-19 en Perú, contagios por distrito")

maPE.get_root().html.add_child(folium.Element(legend_html))
maPE.get_root().html.add_child(folium.Element(title_html))

### El mapa

<p style='text-align: justify; font-size:12px;'>
    Finalmente, tenemos nuestro mapa interactivo. En el notebook los caracteres del español no se muestran correctamente; sin embargo, en los navegadores sí lo hacen. Cada uno de los círculos muestra el número de contagiados en el área. Los markers representan un distrito y los colores van de acuerdo con la leyenda.

In [18]:
maPE.save("COVID19.html")
maPE

<p style='text-align: justify; font-size:12px;'>
    <b>Para ver el mapa en una ventana nueva:

In [19]:
import webbrowser
webbrowser.open("COVID19.html",new=2)

True

**Notas finales**

<p style='text-align: justify; font-size:12px;'>
    Dado que estamos utilizando Selenium para recuperar la información directamente desde el portal del Ministerio de Salud (MINSA) en <a href="https://www.datosabiertos.gob.pe/dataset/casos-positivos-por-covid-19-ministerio-de-salud-minsa">datosabiertos.gob.pe</a>, ello solo será posible siempre y cuando no se modifiquen <code>link</code> ni <code>xpath</code>. Si estos fueran a cambiar, deberían ser ajustados manualmente.